In [1]:
import os
os.chdir('/home/reffert/DeepAR_InfluenzaForecast')
from PythonFiles.Configuration import Configuration
from PythonFiles.HpTuning import get_data, objectiveFNN
from ray.air import session
from ray import tune
data_splits_dict = get_data(truncate=False, with_features=True)[0]

In [ ]:
train = data_splits_dict["with_features_2001"][0]
test = data_splits_dict["with_features_2001"][1]
configuration = Configuration()
hp_search_space = {
    "num_hidden_dimensions": tune.grid_search([[140,140],[180,180]]),
    "context_length":tune.grid_search([32, 52, 64, 104, 208]),
    "epochs":tune.grid_search([75, 150, 200]),
    "prediction_length" : tune.grid_search([4]), 
    "num_batches_per_epoch":tune.grid_search([20, 60]),
    "batch_normalization":tune.grid_search([True, False]),
    "batch_size":tune.grid_search([64, 124]),
    }

tuner = tune.Tuner(
    tune.with_parameters(objectiveFNN, train=train, test=test, configuration=configuration),
    tune_config=tune.TuneConfig(
        num_samples = 5,
        metric="mean_WIS",
        mode="min",
        max_concurrent_trials=14,
    ),
    param_space=hp_search_space,
)
results = tuner.fit()

print("Best hyperparameters found were: ", results.get_best_result().config)

results_df = results.get_dataframe()
print(results_df)
results_df.to_csv("FNN_Hyperparameter_results_01_06.csv")

  0%|          | 0/20 [00:00<?, ?it/s]m 
